In [17]:
import pandas as pd
import numpy as np
import requests
import re
import math
from bs4 import BeautifulSoup, NavigableString
import locale
import time
locale.setlocale(locale.LC_ALL, 'nl_NL.UTF-8')

import aux

from selenium import webdriver
from selenium.webdriver.common.by import By

In [18]:
pd.read_table("../data/trial_small/metadata_in.csv", index_col = 0).head(5)

,Instantie,Datum uitspraak,Datum publicatie,Zaaknummer,Rechtsgebieden,Bijzondere kenmerken,Inhoudsindicatie,Vindplaatsen,Unnamed: 9,Wetsverwijzingen,Formele relaties
0,Rechtbank Den Haag,09-11-2022,09-11-2022,C-09-632745-KG ZA 22-669,Civiel recht,Kort geding,Kort geding. Vordering om Staat te verbieden e...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
1,Rechtbank Den Haag,09-11-2022,09-11-2022,C/09/609567 / HA ZA 21-300,Civiel recht,Bodemzaak\nEerste aanleg - meervoudig,Beroep op volledige nadeelscompensatie (voor o...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
2,Rechtbank Den Haag,09-11-2022,09-11-2022,C/09/601949 / HA ZA 20-1067,Civiel recht,Bodemzaak\nEerste aanleg - meervoudig,Geschil over het uit de VVD-fractie zetten van...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
3,Rechtbank Den Haag,07-11-2022,08-11-2022,C/09/623033 / FA RK 21-8848,Personen- en familierecht,Beschikking,Vaststellen van een verlengde onderhoudsplicht...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
4,Rechtbank Rotterdam,04-11-2022,04-11-2022,C/10/645656 / KG ZA 22-837,Civiel recht,Kort geding,Kort geding; vordering tegen bestuurdersaanspr...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN


In [19]:
metadata_out = pd.read_table("../data/trial_small/metadata_out.csv", index_col = 0)
metadata_out.head(5)

,index,Datum uitspraak:,Datum publicatie:,Rechtsgebieden:,Bijzondere kenmerken:,Vindplaatsen:,Inhoudsindicatie:,Formele relaties:
0,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Kort geding. Vordering om Staat te verbieden e...,NaN
1,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,"Bodemzaak, Eerste aanleg - meervoudig",Rechtspraak.nl,Beroep op volledige nadeelscompensatie (voor o...,NaN
2,https://uitspraken.rechtspraak.nl/inziendocume...,09-11-2022,09-11-2022,Civiel recht,"Bodemzaak, Eerste aanleg - meervoudig",Rechtspraak.nl,Geschil over het uit de VVD-fractie zetten van...,NaN
3,https://uitspraken.rechtspraak.nl/inziendocume...,07-11-2022,08-11-2022,Personen- en familierecht,Beschikking,Rechtspraak.nl,Vaststellen van een verlengde onderhoudsplicht...,NaN
4,https://uitspraken.rechtspraak.nl/inziendocume...,04-11-2022,04-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Kort geding; vordering tegen bestuurdersaanspr...,NaN


In [20]:
# Coalesce the columns for Beslissing and De Beslissing
text = pd.read_table("../data/trial_small/data.csv", index_col = 0)
text['De beslissing'] = text['De beslissing'].combine_first(text['Beslissing'])
text = text.drop(['Beslissing'], axis = 1)

# Coalesce the columns for De Beoordeling and De beoordeling van het geschil
text['De beoordeling'] = text['De beoordeling'].combine_first(text['De beoordeling van het geschil'])
text = text.drop(['De beoordeling van het geschil'], axis = 1)

In [21]:
# Merge two dataframes with text and with URL's: need only metadata_out and text
data = pd.merge(metadata_out, text, left_index=True, right_index=True)


In [22]:
data = data[data['Rechtsgebieden:'] == 'Civiel recht'].reset_index(drop = True)

In [25]:
# This is the major loop
out = dict()
driver = webdriver.Chrome()

for case in range(10):#len(data)):
    out[case] = dict()
    url = data['index'].iloc[case]
    driver.get(url)
    
    time.sleep(3)
    
    # The following is very important
    # Limit the information to only the div in class uitspraak
    # Otherwise - more noise 
    page = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(page, 'html.parser')
    soup = soup.find('div', {'class':'uitspraak'})
    
    geschil = data['Het geschil'].iloc[case]
    beslissing = data['De beslissing'].iloc[case]

    lawyers = aux.extract_lawyers(soup)
    pl_def = aux.extract_plaintiffs_defendants(soup)
            
    if isinstance(geschil, str):
        bedrag = aux.extract_numbers(geschil) 
    else:
        bedrag = None
        
    if isinstance(beslissing, str):
        uitkomst = aux.extract_uitkomst(beslissing) 
    else:
        uitkomst = None
    
    out[case]['lawyers'] = lawyers
    out[case]['pl_def'] = pl_def
    out[case]['bedrag'] = bedrag
    out[case]['uitkomst'] = uitkomst
    
driver.close()

In [26]:
out

{0: {'lawyers': ['advocaten mrs. A.M. Seebregts en H.J.D. de Boer te Rotterdam en mr. T.Y Tsang te Den Haag,',
   'advocaat mr. J. Perenboom te Den Haag.'],
  'pl_def': (['in de zaak van',
    '\n          [eiser] ',
    'te [plaats 1] ,',
    'eiser,',
    'advocaten mrs. A.M. Seebregts en H.J.D. de Boer te Rotterdam en mr. T.Y Tsang te Den Haag,'],
   ['tegen:',
    'de Staat der Nederlanden, meer specifiek het Ministerie van Justitie en Veiligheid,',
    'gedaagde,',
    'advocaat mr. J. Perenboom te Den Haag.']),
  'bedrag': (set(), None, None),
  'uitkomst': ({0: None,
    1: <re.Match object; span=(1, 36), match='wijst de vorderingen van [eiser] af'>,
    2: None,
    3: None,
    4: None},
   {},
   {},
   {0: set(), 1: set(), 2: {676.0, 1016.0, 1692.0}, 3: set(), 4: set()},
   {},
   {})},
 1: {'lawyers': ['advocaat mr. P.D. Olden te Amsterdam,',
   'advocaat mr. R.W. Veldhuis te Den Haag.'],
  'pl_def': (['in de zaak van',
    'de vereniging met volledige rechtsbevoegdheid',
 

In [ ]:
# Find stuff with reconventie in it to test
#text['De beslissing'].str.contains('reconventie')[50:100]